In [ ]:
library(tidyverse)
library(stringr) 
library(plotly)
library(cowplot)

In [ ]:
##############################################################################

##########################     INPUT PARAMETERS    ###########################

##############################################################################



# filetype = # "ratio_all_germline_singletons_excluded_tumor_4_cs" # # "ratio_singletons_excluded_tumor_4_cs" # "ratio" 
filetype = "ratio"

# input_folder =   "ci5_cs1e9" # "all_germline_ci5_cs1e9" # "all_germline_filtered_bams_tumor_ci5_cs1e9" # "all_germline_filtered_bams_tumor_subtract_other_tumor_ci5_cs1e9", all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA
# all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA # de_novo_germline # all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA_subtracted_kmers_not_seen
# all_germline_filtered_bams_tumor_ci5_cs1e9_well_mapping_subtracted # de_novo_germline # de_novo_germline_filtered_cfDNA # de_novo_germline_reference
input_folder = "final_pipeline_NEW"

# output_folder =  "ci5_cs1e9" # "union_germline_ci4_cs1e9" # "union_germline_filtered_tumor_bams_ci5_cs1e9" # "union_germline_filtered_tumor_bams_subtract_other_tumors_ci5_cs1e9"
# union_germline_filtered_tumor_bams_ci5_cs1e9_filtered_cfDNA # de_novo_germline # union_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA_subtracted_kmers_not_seen
# union_germline_filtered_tumor_ci5_cs1e9_well_mapping_subtracted # de_novo_germline # de_novo_germline_filtered_cfDNA # de_novo_germline_reference
output_folder = "final_pipeline_pII"

input_phase = "phaseII"

low_qual_sample <- read.table("../low_qual_sample.txt")
low_qual_sample <- as.character(low_qual_sample[[1]])

In [ ]:
paramspace = read.csv("../data/metadata/paramspace_cfDNA_phaseII_missing_removed.csv")

paramspace  <- paramspace %>% rename(patient_id = pt_id)
rm_pt <- read.csv("../phase2_exclude3.csv", header=FALSE)
rm_pt <- as.character(unlist(c(rm_pt[1,])))
paramspace <- paramspace %>% filter(!patient_id %in% rm_pt)

head(paramspace)

In [ ]:
SAMPLES = paramspace["patient_id"]
FOLDERS = paramspace["cfDNA_folder"]

In [ ]:
nrow(paramspace)

In [ ]:
#paramspace["big_ratio"] <- NA
#paramspace["big_ratio_CI_lower"] <- NA
#paramspace["big_ratio_CI_upper"] <- NA

paramspace["small_ratio"] <- NA
paramspace["small_ratio_CI_lower"] <- NA
paramspace["small_ratio_CI_upper"] <- NA

for (row in 1:nrow(paramspace)){
    pt = paramspace[row, "patient_id"]
    #print(pt)
    fd = paramspace[row, "cfDNA_folder"]
    #big_r = read.csv(paste("../data/phase_II_patients/", pt, "/" , fd, "/", input_folder, "/", "big_", filetype, ".csv", sep = ""))
    small_r = read.csv(paste("../data/phase_II_patients/", pt, "/", fd, "/", input_folder, "/", "small_", filetype, ".csv", sep = ""))
    #print(big_r[[1]])
    #print(small_r[[1]])
    #paramspace[row, "big_ratio"] = big_r$ratio
    #paramspace[row, "big_ratio_CI_lower"] = big_r$lower_CI
    #paramspace[row, "big_ratio_CI_upper"] = big_r$upper_CI

    
    paramspace[row, "small_ratio"] = small_r$ratio
    paramspace[row, "small_ratio_CI_lower"] = small_r$lower_CI
    paramspace[row, "small_ratio_CI_upper"] = small_r$upper_CI
}

In [ ]:
res <- paramspace
head(res)

In [ ]:
#res <- res %>% separate(cfDNA_folder, c("sampleID", "sample_type", "sample_number"))
res <- res %>% 
  mutate(tmp_chunks = stringr::str_split(cfDNA_folder, stringr::fixed("_"),  n = 3)) %>%
  mutate(sampleID = map_chr(tmp_chunks, 1),
         sub_value = map_chr(tmp_chunks, 2), 
         sub_val = map_chr(tmp_chunks, 3)) %>%
  select(-c(tmp_chunks, sub_value, sub_val))

head(res)
dim(res)

In [ ]:
sample_timepoint <- read.csv("../data/metadata/clin_data/sample_timepoint_formatted.csv")

In [ ]:
head(sample_timepoint)
sample_timepoint_days <- sample_timepoint %>% filter(phase == input_phase)
sample_timepoint_days <- sample_timepoint_days %>% select(sampleID, sample_timepoint_days_since_OP)
sample_timepoint_days %>% filter(is.na(sample_timepoint_days_since_OP))
dim(sample_timepoint_days)
head(sample_timepoint_days)

In [ ]:
res$sampleID <- as.character(res$sampleID)
res <- left_join(res, sample_timepoint_days, by = c("sampleID"))
head(res)
dim(res)

In [ ]:
res %>% filter(is.na(sample_timepoint_days_since_OP))

In [ ]:
clinical_data <- read.csv("../data/metadata/clin_data/clinical_data_formatted.csv")
clinical_data_relapse <- clinical_data %>% select(patient_id, time_to_relapse_days, adjuvant_chemo_start_days, adjuvant_chemo_end_days)
head(clinical_data_relapse)

In [ ]:
res <-left_join(res, clinical_data_relapse, by="patient_id")
head(res)
dim(res)

In [ ]:
res_min <- NULL
for (pt in unique(res$patient_id)){
    res_pt <- res %>% filter(patient_id == pt)
    res_pt$sample_timepoint_days_since_OP <- as.numeric(res_pt$sample_timepoint_days_since_OP)
    
    # order based on sample timepoint
    res_pt <- res_pt %>% arrange(sample_timepoint_days_since_OP)
    #head(res_pt)
    
    # pick first 2
    res_pt <- res_pt[1:2, ]
    #print(res_pt)
    
    res_pt <- res_pt %>% mutate(sample_label = c("pre_op", "post_op"))
    #print(res_pt)
    
    if (as.numeric(res_pt[1, "sample_timepoint_days_since_OP"]) > 0){
        # test expression
        next 
    } 
    
    # save to res min
    res_min <- rbind(res_min, res_pt)
    
    }

res_min %>% select(patient_id, cfDNA_folder, small_ratio, sample_timepoint_days_since_OP, sample_label) 
dim(res_min)

In [ ]:
res_min <- res_min %>% mutate(pre_post_time = ifelse(sample_label == "pre_op", 0, sample_timepoint_days_since_OP))

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)

In [ ]:
res_filtered <- res_min %>% select(patient_id, cfDNA_folder, small_ratio, sample_timepoint_days_since_OP, sample_label) #big_upper_ratio,

In [ ]:
normalized_res <- data.frame(matrix(ncol = 6, nrow = 0))
colnames(normalized_res) <- c("patient_id", "cfDNA_folder", "small_ratio", "sample_timepoint_days_since_OP", "sample_label", "small_ratio_normalized")


In [ ]:
patients = as.character(unique(res_filtered$patient_id))

In [ ]:
for (i in patients){
    res_pt <- res_filtered %>% filter(patient_id == i)
    
    for (x in res_pt$cfDNA_folder){
        sample_label_check <- res_pt %>% filter(cfDNA_folder == x)
        
        if (sample_label_check$sample_label == "pre_op"){
            #pre_val_big = sample_label_check$big_ratio
            #pre_val_jaccard = sample_label_check$jaccard
            pre_val_small = sample_label_check$small_ratio}}
    
    for (j in res_pt$cfDNA_folder){
        normalized_line <- res_pt %>% filter(cfDNA_folder == j)
        #print(normalized_line)
        normalized_line <- normalized_line %>% mutate(small_ratio_normalized = small_ratio/pre_val_small) #
        #print(normalized_line)
        normalized_res[nrow(normalized_res)+1,] = c(i, j, normalized_line[3:6])
    
    }}

In [ ]:
relapse <- read.csv("../data/metadata/clin_data//clinical_data_formatted.csv")
relapse <- relapse %>% filter(C2i_phase == "phaseII") %>% select(patient_id, is_relapse..0.no.1.yes., time_to_relapse_days)
relapse_label <- relapse %>% mutate(Relapse_label = ifelse(is_relapse..0.no.1.yes. == 0, "No_relapse", "Relapse")) %>% select(patient_id, Relapse_label)
head(relapse_label)

In [ ]:
normalized_res <- left_join(normalized_res, relapse_label, by="patient_id")

In [ ]:
normalized_res <- normalized_res %>% mutate(pre_post_time = ifelse(sample_label == "pre_op", 0, sample_timepoint_days_since_OP))
head(normalized_res)

In [ ]:
#quick_plot_big_r_normalized <- ggplot(normalized_res) + geom_line(aes(x = pre_post_time, y = log10(big_ratio_normalized), color = Relapse_label, group=patient_id)) + 
#                                      geom_point(aes(x = pre_post_time, y = log10(big_ratio_normalized), color = Relapse_label)) 
#

In [ ]:
#quick_plot_big_r_normalized

In [ ]:
#ggsave(
#  paste("../plotting_results/", output_folder, "/pre_post_plots/Normalized_Big_", filetype, ".png", sep = ""),
#  quick_plot_big_r_normalized, dpi = "print")

In [ ]:
#quick_plot_jaccard_normalized <- ggplot(normalized_res) + geom_line(aes(x = pre_post_time, y = log10(jaccard_normalized), color = Relapse_label, group=patient_id)) + 
#                                      geom_point(aes(x = pre_post_time, y = log10(jaccard_normalized), color = Relapse_label)) + theme(text = element_text(size = 20)) 
#ggsave(
#  paste("../plotting_results/", output_folder, "/pre_post_plots/Normalized_Jaccard_", filetype, ".png", sep = ""),
#  quick_plot_jaccard_normalized, dpi = "print")

In [ ]:
#quick_plot_jaccard_normalized

In [ ]:
quick_plot_small_r_normalized <- ggplot(normalized_res %>% filter(patient_id != low_qual_sample)) + geom_line(aes(x = pre_post_time, y = log10(small_ratio_normalized), color = Relapse_label, group=patient_id), size = 0.5) + 
                                      geom_point(aes(x = pre_post_time, y = log10(small_ratio_normalized), color = Relapse_label), size = 1) + 
    theme_minimal() + 
    theme(text = element_text(size = 15)) + 
    xlab("Sample time point (days since surgery)")  + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))
    #+ 
    #ggtitle("Fraction of unique tumor k-mers found in cfDNA,\nnormalized by the pre-op sample (low quality sample removed)")


In [ ]:
quick_plot_small_r_normalized

In [ ]:
quick_plot_small_r_normalized <- ggplot(normalized_res %>% filter(patient_id != low_qual_sample)) + geom_line(aes(x = pre_post_time, y = log10(small_ratio_normalized), color = Relapse_label, group=patient_id), size = 0.5) + 
                                      geom_point(aes(x = pre_post_time, y = log10(small_ratio_normalized), color = Relapse_label), size = 1) + 
    theme_minimal() + 
    theme(text = element_text(size = 12),
          strip.text.x = element_blank()) + 
    xlab("Sample time point (days since surgery)")  + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"), name = "Relapse label") + 
    facet_wrap(~Relapse_label, ncol = 2, scales = "free_x")
    #+ 
    #ggtitle("Fraction of unique tumor k-mers found in cfDNA,\nnormalized by the pre-op sample (low quality sample removed)")

options(repr.plot.width=9, repr.plot.height=6)
quick_plot_small_r_normalized

In [ ]:
ggsave(
  paste("../plotting_results/", output_folder, "/pre_post_plots/Normalized_Small_", filetype, ".png", sep = ""),
  quick_plot_small_r_normalized, dpi = "print")

In [ ]:
normalized_res

In [ ]:
quick_plot_big_r_ <- ggplot(normalized_res) + geom_line(aes(x = pre_post_time, y = big_ratio, color = Relapse_label, group=patient_id)) + 
                                      geom_point(aes(x = pre_post_time, y = big_ratio, color = Relapse_label)) + theme(text = element_text(size = 20)) 
ggsave(
  paste("../plotting_results/", output_folder, "/pre_post_plots/Big_", filetype, ".png", sep = ""),
  quick_plot_big_r_, dpi = "print")

In [ ]:
quick_plot_big_r_

In [ ]:
quick_plot_small_r <- ggplot(normalized_res) + geom_line(aes(x = pre_post_time, y = small_ratio, color = Relapse_label, group=patient_id)) + 
                                      geom_point(aes(x = pre_post_time, y = small_ratio, color = Relapse_label)) + theme(text = element_text(size = 20)) 
#ggsave(
#  paste("../plotting_results/", output_folder, "/pre_post_plots/Small_", filetype, ".png", sep = ""),
#  quick_plot_small_r, dpi = "print")

In [ ]:
quick_plot_small_r